In [6]:
import pandas as pd
import numpy as np


credit_card_data = pd.read_csv('/Volumes/LACIE_SHARE/python_scripts/Data/credit_card.csv')



In [2]:
credit_card_data = pd.read_csv('/Volumes/LACIE_SHARE/python_scripts/Data/credit_card.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [57]:
print(np.max(credit_card_data['default payment next month']))
print(np.sum(credit_card_data['default payment next month']))

features = credit_card_data.drop('default payment next month', axis = 1)
target = credit_card_data['default payment next month']


### sex, education, and marriage are numeric. Are they binary?
print(np.max(credit_card_data['marriage']))

print(np.max(credit_card_data['education']))


1
6636


KeyError: 'marriage'

In [15]:
## Import the sklearn modules

from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split 

X_train_org, X_test_org, y_train, y_test = train_test_split(features, target, random_state = 0, test_size = 0.4)
## Our dataset has 30000 observations, so we can get away with a larger test set.

## We'll do scaling after the train/test split to avoid data leakage

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.fit_transform(X_test_org)



In [39]:
## K nearest neighbors

from sklearn.neighbors import KNeighborsClassifier

num_knn = 30

train_score_array = []
test_score_array = []

for k in range (1, num_knn):
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    train_score_array.append(knn.score(X_train, y_train))
    test_score_array.append(knn.score(X_test, y_test))
    print("k = {}, Training Accuracy: {}, Testing Accuracy: {}".\
          format(k, np.round(knn.score(X_train, y_train), decimals = 4),\
                 np.round(knn.score(X_test, y_test), decimals = 4)))

k: 1, Training Accuracy: 1.0, Testing Accuracy: 0.7237
k: 2, Training Accuracy: 0.8596, Testing Accuracy: 0.786
k: 3, Training Accuracy: 0.8618, Testing Accuracy: 0.7768
k: 4, Training Accuracy: 0.8361, Testing Accuracy: 0.7946
k: 5, Training Accuracy: 0.8372, Testing Accuracy: 0.7931
k: 6, Training Accuracy: 0.8271, Testing Accuracy: 0.8009
k: 7, Training Accuracy: 0.8268, Testing Accuracy: 0.8026
k: 8, Training Accuracy: 0.8224, Testing Accuracy: 0.8051
k: 9, Training Accuracy: 0.8228, Testing Accuracy: 0.8053
k: 10, Training Accuracy: 0.8197, Testing Accuracy: 0.8061
k: 11, Training Accuracy: 0.8206, Testing Accuracy: 0.8077
k: 12, Training Accuracy: 0.817, Testing Accuracy: 0.8078
k: 13, Training Accuracy: 0.8177, Testing Accuracy: 0.8082
k: 14, Training Accuracy: 0.8151, Testing Accuracy: 0.8082
k: 15, Training Accuracy: 0.8157, Testing Accuracy: 0.8092
k: 16, Training Accuracy: 0.8143, Testing Accuracy: 0.8093
k: 17, Training Accuracy: 0.8144, Testing Accuracy: 0.809
k: 18, Train

In [40]:
## The optimal k value happens when our train and test value have their minimum distance. 

difference_array = []

for i in range(0, num_knn-1):
    difference_array.append(np.abs(train_score_array[i] - test_score_array[i]))

print('our minimum distance occurred at k = {}'.\
      format(difference_array.index(np.amin(difference_array)) + 1)) 
#adding one because our knn testing started at 1, so the first value will be 
    
print('our min distance was {}'.format(np.amin(difference_array)))




our minimum distance occurred at k = 26
our min distance was 0.0005277777777777937


In [56]:
## Grid search algorithm using https://medium.com/@erikgreenj/k-neighbors-classifier-with-gridsearchcv-basics-3c445ddeb657
## This search adjusts weights and method as well as k
## (Dr Snell's program did not adjust anything but k so his grid search was redundant, I think)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

k_vals = range(21, 40, 2) ## go by odd numbers because even k numbers can have ties. 

param_grid = {'n_neighbors': k_vals,
             'weights': ['uniform', 'distance'], ## Can comment out weights and metric if it takes too long
             'metric': ['euclidean', 'manhattan']
             }

grid_search = GridSearchCV(KNeighborsClassifier(), 
                           param_grid, 
                           verbose = 1,
                           cv = 15, 
                           n_jobs = -1, ## use all my cores
                           return_train_score = True)

grid_search.fit(X_train, y_train) ## Our first model is made



Fitting 15 folds for each of 40 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 33.4min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 48.4min finished


GridSearchCV(cv=15, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(21, 40, 2),
                         'weights': ['uniform', 'distance']},
             return_train_score=True, verbose=1)

In [61]:
#Best score and parameters
print("Best score: {:.4f}".format(grid_search.best_score_))
print("Best parameters: {}".format(grid_search.best_params_))

## Accuracy
knn = KNeighborsClassifier(metric = 'euclidean', n_neighbors = 35, weights = 'distance')
knn.fit(X_train, y_train)
print("Training Accuracy:",knn.score(X_train, y_train)) #Accuracy of the model when training.
print("Testing Accuracy:", knn.score(X_test, y_test) )#Accuracy of the test.


Best score: 0.8072
Best parameters: {'metric': 'euclidean', 'n_neighbors': 35, 'weights': 'distance'}
Training Accuracy: 1.0
Testing Accuracy: 0.81


In [62]:
## Support Vector Classifier

from sklearn.svm import SVC

from sklearn.svm import SVC  

linear_svclassifier = SVC(kernel='linear')  
linear_svclassifier.fit(X_train, y_train) 

poly_svclassifier = SVC(kernel='poly')  
poly_svclassifier.fit(X_train, y_train) 

rbf_svclassifier = SVC(kernel='rbf')  
rbf_svclassifier.fit(X_train, y_train) 

sigmoid_svclassifier = SVC(kernel='sigmoid')  
sigmoid_svclassifier.fit(X_train, y_train) 




### We now have 4 SVCs to choose from 

## We can do a grid search for each one






ValueError: Precomputed matrix must be a square matrix. Input is a 18000x24 matrix.

In [36]:
## We want the results of our individual models to be poorly correlated but also better than a naive. 


0.0005277777777777937
0.0019444444444444153


In [ ]:
""" At a minimum you should report F1 scores, ROC curve, and AUC, 
 but you might also want to consider using  accuracy scores, 
 confusion matrices, recall, precision, sensitivity, specificity, 
false positives, false negatives, Matthews correlation coefficient etc """

ROC = 
F1 = 
AUC = 
accuracy = 
conf_matrix = 
recall = 
precision = 
sensitivity = 
specificty = 
f_pos = 
f_neg = 
matt_cor = 